# Deploy our ML Model

**SageMaker Studio Kernel**: Data Science

In this exercise you will do:
 - Run a Preprocessing Job using Amazon SageMaker Processing Job
 - Run a Tensorflow Training Job using Amazon SageMaker Training Job
 - Register a new version of the trained model in the Amazon SageMaker Model Registry

***

## Part 1/4 - Setup
Here we'll import some libraries and define some variables.

In [ ]:
! pip install s3fs

### Import required modules

In [ ]:
import boto3
from botocore.exceptions import ClientError
from datetime import datetime
import logging
from sagemaker import get_execution_role
from sagemaker.deserializers import JSONLinesDeserializer
from sagemaker.model_monitor import DataCaptureConfig
import sagemaker.session
from sagemaker.tensorflow import TensorFlowModel
import traceback

In [ ]:
s3_client = boto3.client("s3")
sagemaker_client = boto3.client("sagemaker")

In [ ]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

***

## Part 2/4 - Model Package Definition
During this steps, we are retrieving model informations from the Amazon SageMaker Model Registry

### Get Approved Model Packages

This method can be used for returning the last approved model from the specified model package group

In [ ]:
model_package_group = "ml-end-to-end-group"

In [ ]:
try:
    # Get the latest approved model package
    response = sagemaker_client.list_model_packages(
        ModelPackageGroupName=model_package_group,
        ModelApprovalStatus="Approved",
        SortBy="CreationTime",
        SortOrder="Descending",
        MaxResults=1,
    )
    approved_packages = response["ModelPackageSummaryList"]

    # Return error if no packages found
    if len(approved_packages) == 0:
        error_message = ("No approved ModelPackage found for ModelPackageGroup: {}".format(model_package_group))
        LOGGER.error("{}".format(error_message))

        raise Exception(error_message)

    model_package = approved_packages[0]
    LOGGER.info("Identified the latest approved model package: {}".format(model_package))
except ClientError as e:
    stacktrace = traceback.format_exc()
    error_message = e.response["Error"]["Message"]
    LOGGER.error("{}".format(stacktrace))

    raise Exception(error_message)

### List Model Packages

This method can be used for listing all the registered models in a Model Package Group

In [ ]:
model_package_arn = model_package["ModelPackageArn"]

In [ ]:
try:
    model_package = sagemaker_client.describe_model_package(
        ModelPackageName=model_package_arn
    )

    LOGGER.info("{}".format(model_package))

    if len(model_package) == 0:
        error_message = ("No ModelPackage found for: {}".format(model_package_arn))
        LOGGER.error("{}".format(error_message))

        raise Exception(error_message)
except ClientError as e:
    stacktrace = traceback.format_exc()
    error_message = e.response["Error"]["Message"]
    LOGGER.error("{}".format(stacktrace))

    raise Exception(error_message)

***

## Part 3/4 - Deploy an Amazon SageMaker Endpoint
Here we are deploying an Amazon SageMaker Endpoint by using the ML model taken from the Model Registry

In [ ]:
region = boto3.session.Session().region_name
role_name = "mlops-sagemaker-execution-role"
role = "arn:aws:iam::{}:role/{}".format(boto3.client('sts').get_caller_identity().get('Account'), role_name)

kms_account_id = boto3.client('sts').get_caller_identity().get('Account')

kms_alias = "ml-kms"

bucket_artifacts = ""
bucket_inference = ""

inference_artifact_path = "artifact/inference"
inference_artifact_name = "sourcedir.tar.gz"
inference_instance_count = 1
inference_instance_type = "ml.m5.xlarge"

model_package_group = "ml-end-to-end-group"


monitoring_output_path = "data/monitoring/captured"

training_framework_version = 2.4

In [ ]:
kms_key = "arn:aws:kms:{}:{}:alias/{}".format(region, kms_account_id, kms_alias)

In [ ]:
boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client("sagemaker")
runtime_client = boto_session.client("sagemaker-runtime")

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_runtime_client=runtime_client,
    default_bucket=bucket_inference
)

### Compress source code for installing additional python modules

In [ ]:
! ./../algorithms/buildspec.sh inference $bucket_artifacts

In [ ]:
inference_source_dir = "s3://{}/{}/{}".format(
    bucket_inference,
    inference_artifact_path,
    inference_artifact_name
)

### Create SageMaker model

This method can be used for creating a SageMaker model

In [ ]:
try:
    model = TensorFlowModel(
        entry_point="inference.py",
        framework_version=str(training_framework_version),
        source_dir=inference_source_dir,
        model_data=model_package["InferenceSpecification"]["Containers"][0]["ModelDataUrl"],
        model_kms_key=kms_key,
        role=role,
        sagemaker_session=sagemaker_session
    )
except Exception as e:
    stacktrace = traceback.format_exc()
    LOGGER.error("{}".format(stacktrace))

    raise e

### Deploy a SageMaker Endpoint

Lets deploy the endpoint. If we want to update an existing endpoint, we have to create a new endpoint configuration defined in the method below

In [ ]:
def update_model(session, model_name, model_package_group_name, env, inference_instance_count, inference_instance_type):
    try:
        LOGGER.info("Updating endpoint configuration {}".format(model_package_group_name + "-" + env))

        endpoint_config_name = session.create_endpoint_config(
            name="{}-{}-{}".format(model_package_group_name, env, datetime.today().strftime('%Y-%m-%d-%H-%M-%S')),
            model_name=model_name,
            initial_instance_count=inference_instance_count,
            instance_type=inference_instance_type
        )

        response = sagemaker_client.update_endpoint(
            EndpointName=model_package_group_name + "-" + env,
            EndpointConfigName=endpoint_config_name
        )

        LOGGER.info("Update endpoint {}-{}".format(model_package_group_name, env))
        LOGGER.info(response)

    except Exception as e:
        stacktrace = traceback.format_exc()
        LOGGER.info("{}".format(stacktrace))

        raise e

In [ ]:
try:
    model.deploy(
        endpoint_name=model_package_group + "-dev",
        initial_instance_count=inference_instance_count,
        instance_type=inference_instance_type,
        update_endpoint=True,
        data_capture_config=DataCaptureConfig(
                enable_capture=True,
                sampling_percentage=100,
                json_content_types=["application/jsonlines"],
                destination_s3_uri="s3://{}/{}".format(bucket_inference, monitoring_output_path))
    )
except ClientError as e:
    stacktrace = traceback.format_exc()
    LOGGER.info("{}".format(stacktrace))

    model_name = get_deployed_model()

    update_model(session, model_name, model_package_group_name, env, inference_instance_count, inference_instance_type)

### Test the SageMaker Endpoint

In [ ]:
model_package_group = "ml-end-to-end-group"

In [ ]:
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

predictor = TensorFlowPredictor(
    endpoint_name=model_package_group + "-dev",
    model_name="saved_model",
    model_version=1,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    serializer=JSONLinesSerializer(),
    deserializer=JSONLinesDeserializer()
    
)

In [ ]:
inputs = [{"features": ["Sei disgustoso"]}]

result = predictor.predict(inputs)

LOGGER.info("{}".format(result))

## Part 3/4 - Monitoring
Here we are creating monitoring jobs for extracting metrics from our SageMaker Endpoint

### Create a Baseline for the monitoring job

From our training dataset, let's select the relevant attributes and generate a dataset for baselining. Then we use Amazon SageMaker Model Monitor to suggest a set of baseline constraints and descriptive statistics.

In [ ]:
import pandas as pd
from sagemaker.model_monitor import CronExpressionGenerator, DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from time import gmtime, strftime

In [ ]:
bucket_artifacts = ""
bucket_inference = ""

monitoring_input_files_path = "data/monitoring/input"
processing_output_files_path = "data/output"

In [ ]:
cleaned_data = "s3://{}/{}/processed_data.csv".format(bucket_artifacts, processing_output_files_path)

monitoring_data = "s3://{}/{}/monitoring_data.csv".format(bucket_artifacts, monitoring_input_files_path)

columns = ["text", "Sentiment"]

df = pd.read_csv(cleaned_data, usecols=columns)
df.to_csv(monitoring_data, index=None)

In [ ]:
baseline_input_path = "s3://{}/{}".format(bucket_inference, monitoring_input_files_path)
baseline_output_path = "s3://{}/data/monitoring/output".format(bucket_inference)

Please note that running the baselining job will require 8-10 minutes. In the meantime, you can take a look at the Deequ library, used to execute these analyses with the default Model Monitor container: https://github.com/awslabs/deequ

In [ ]:
monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.c5.4xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

In [ ]:
monitor.suggest_baseline(
    baseline_dataset=baseline_input_path,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_output_path,
    wait=True
)

### Create Monitoring Scheduler

Here we are creating our monitoring scheduler. It will execute monitoring jobs with hourly schedule execution. When we create the schedule, we can also specify two scripts that will preprocess the records before the analysis takes place and execute post-processing at the end. For this example, we are not going to use a record preprocessor, and we are just specifying a post-processor that outputs some text for demo purposes.

In [ ]:
!pygmentize ./../algorithms/monitoring/src/preprocess.py

In [ ]:
!pygmentize ./../algorithms/monitoring/src/postprocess.py

In [ ]:
monitoring_code_prefix = "artifact/monitoring"

boto3.Session().resource("s3").Bucket(bucket_artifacts).Object(monitoring_code_prefix + "/preprocess.py").upload_file("./../algorithms/monitoring/src/preprocess.py")
boto3.Session().resource("s3").Bucket(bucket_artifacts).Object(monitoring_code_prefix + "/postprocess.py").upload_file("./../algorithms/monitoring/src/postprocess.py")

preprocessor_path = "s3://{}/{}/preprocess.py".format(bucket_artifacts, monitoring_code_prefix)
postprocessor_path = "s3://{}/{}/postprocess.py".format(bucket_artifacts, monitoring_code_prefix)

LOGGER.info(preprocessor_path)
LOGGER.info(postprocessor_path)

reports_path = "s3://{}/data/monitoring/reports".format(bucket_inference)

LOGGER.info(reports_path)

In [ ]:
endpoint_name = predictor.endpoint_name

mon_schedule_name = "" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=endpoint_name,
    record_preprocessor_script=preprocessor_path,
    post_analytics_processor_script=postprocessor_path,
    output_s3_uri=reports_path,
    statistics=monitor.baseline_statistics(),
    constraints=monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True
)

In [ ]:
desc_schedule_result = monitor.describe_schedule()
desc_schedule_result

### Delete Scheduler

Once the schedule is created, it will kick of jobs at specified intervals. Note that if you are kicking this off after creating the hourly schedule, you might find the executions empty. You might have to wait till you cross the hour boundary (in UTC) to see executions kick off. Since we don't want to wait for the hour in this example we can delete the schedule and use the code in next steps to simulate what will happen when a schedule is triggered, by running an Amazon SageMaker Processing Job.

In [ ]:
monitor.delete_monitoring_schedule()

### Manual monitoring execution

In oder to trigger the execution manually, we first get all paths to data capture, baseline statistics, baseline constraints, etc. Then, we use a utility fuction, defined in monitoringjob_utils.py, to run the processing job.

In [ ]:
import os
import sys

In [ ]:
sys.path.insert(0, os.path.abspath('./../scripts'))

In [ ]:
from monitoringjob_utils import run_model_monitor_job_processor

In [ ]:
bucket_inference = ""

endpoint_name = predictor.endpoint_name
current_endpoint_capture_prefix = "data/monitoring/captured/{}".format(endpoint_name)

In [ ]:
region = boto3.session.Session().region_name
role_name = "mlops-sagemaker-execution-role"
role = "arn:aws:iam::{}:role/{}".format(boto3.client('sts').get_caller_identity().get('Account'), role_name)

result = s3_client.list_objects(Bucket=bucket_inference, Prefix=current_endpoint_capture_prefix)
capture_files = ["s3://{0}/{1}".format(bucket_inference, capture_file.get("Key")) for capture_file in result.get("Contents")]

data_capture_path = capture_files[len(capture_files) - 1][: capture_files[len(capture_files) - 1].rfind('/')]

baseline_output_path = "s3://{}/data/monitoring/output".format(bucket_artifacts)

statistics_path = baseline_output_path + "/statistics.json"
constraints_path = baseline_output_path + "/constraints.json"

LOGGER.info("Capture Files: ")
LOGGER.info("\n ".join(capture_files))
LOGGER.info(data_capture_path)
LOGGER.info(statistics_path)
LOGGER.info(constraints_path)

In [ ]:
run_model_monitor_job_processor(
    region, 
    "ml.m5.xlarge", 
    role, 
    data_capture_path, 
    statistics_path, 
    constraints_path, 
    reports_path,
    preprocessor_path=preprocessor_path,
    postprocessor_path=postprocessor_path)

### Analysis

Here we are analyzing the report created by our Monitoring Job

In [ ]:
import json
import pandas as pd

In [ ]:
bucket_inference = ""

reports_path = "data/monitoring/reports"

In [ ]:
monitoring_reports_prefix = "{}/{}".format(reports_path, predictor.endpoint_name)

result = s3_client.list_objects(Bucket=bucket_inference, Prefix=monitoring_reports_prefix)

try:
    monitoring_reports = ['s3://{0}/{1}'.format(bucket_inference, capture_file.get("Key")) for capture_file in result.get('Contents')]
    print("Monitoring Reports Files: ")
    print("\n ".join(monitoring_reports))
except:
    print('No monitoring reports found.')

In [ ]:
!aws s3 cp {monitoring_reports[0]} ./../data/monitoring/
!aws s3 cp {monitoring_reports[1]} ./../data/monitoring/
!aws s3 cp {monitoring_reports[2]} ./../data/monitoring/
!aws s3 cp {monitoring_reports[3]} ./../data/monitoring/

In [ ]:
pd.set_option('display.max_colwidth', None)

file = open('./../data/monitoring/constraint_violations.json', 'r')
data = file.read()

violations_df = pd.json_normalize(json.loads(data)['violations'])
violations_df